<a href="https://colab.research.google.com/github/Tyred/TimeSeriesClassification/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [19]:
!pip install -q -U tensorflow-addons

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow import keras

from tensorflow.keras.layers import Conv1D, Activation, Dense, Flatten, \
                                    MaxPooling1D, GlobalAveragePooling1D, Dropout
%matplotlib inline

In [20]:
# Convolutional Autoencoder with MaxPooling:
class CNN(tf.keras.Model):
    def __init__(self, num_classes): #, serie_length):
        super(CNN, self).__init__()
        self.conv_1  = Conv1D(32, 5, activation='swish', padding='same') 
        self.gn_1    = tfa.layers.GroupNormalization(32)
        self.max_1   = MaxPooling1D(2)
        self.conv_2  = Conv1D(64, 5, activation='swish', padding='same')
        self.gn_2    = tfa.layers.GroupNormalization(32)
        self.max_2   = MaxPooling1D(2)
        self.conv_3  = Conv1D(128, 5, activation='swish', padding='same')
        self.gn_3    = tfa.layers.GroupNormalization(32)
        self.max_3   = MaxPooling1D(2)
        self.conv_4  = Conv1D(512, 5, activation='swish', padding='same')
        self.gn_4    = tfa.layers.GroupNormalization(32)
        self.max_4   = MaxPooling1D(2)

        self.flatten = Flatten()
        self.dense_1 = Dense(256, activation='swish')
        self.dropout = dropout = Dropout(0.3)
        self.clf     = Dense(num_classes, activation='softmax')
     
    def call(self, inputs):
        x = self.conv_1(inputs)
        x = self.gn_1(x)
        x = self.max_1(x)

        x = self.conv_2(x)
        x = self.gn_2(x)
        x = self.max_2(x)

        x = self.conv_3(x)
        x = self.gn_3(x)
        x = self.max_3(x)

        x = self.conv_4(x)
        x = self.gn_4(x)
        x = self.max_4(x)

        x = self.flatten(x)
        x = self.dense_1(x)
        x = self.dropout(x)
        return self.clf(x)

    def model(self):
        x = keras.layers.Input(shape=(serie_length, 1))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))

In [21]:
path = 'drive/My Drive/UFSCar/FAPESP/IC/Data/UCRArchive_2018'

dataset = input('Dataset: ')
tr_data = np.genfromtxt(path + "/" + dataset + "/" + dataset + "_TRAIN.tsv", 
                        delimiter="\t",)
te_data = np.genfromtxt(path + "/" + dataset + "/" + dataset + "_TEST.tsv", 
                        delimiter="\t",)

train_labels = np.array([int(label) for label in tr_data[: ,0]]) 
test_labels  = np.array([int(label) for label in te_data[:, 0]])
print("Labels:", np.unique(train_labels))

Dataset: DistalPhalanxOutlineCorrect
Labels: [0 1]


In [22]:
num_classes = len(np.unique(train_labels))

train_data  = tr_data[:, 1:] 
test_data   = te_data[:, 1:]


train_data = tf.cast(train_data, tf.float32)
test_data  = tf.cast(test_data , tf.float32)

train_data = tf.expand_dims(train_data, axis=-1)
test_data  = tf.expand_dims(test_data , axis=-1)

print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)

Train data shape: (600, 80, 1)
Test data shape: (276, 80, 1)


In [23]:
cnn = CNN(num_classes)
cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

batch_size = 32
epochs = 50

history = cnn.fit(train_data, train_labels, 
          epochs=epochs, 
          batch_size=batch_size)

Epoch 1/50
19/19 [==============================] - 2s 8ms/step - loss: 2.8995 - accuracy: 0.5467
Epoch 2/50
19/19 [==============================] - 0s 8ms/step - loss: 0.7292 - accuracy: 0.6467
Epoch 3/50
19/19 [==============================] - 0s 7ms/step - loss: 0.5897 - accuracy: 0.7017
Epoch 4/50
19/19 [==============================] - 0s 7ms/step - loss: 0.5636 - accuracy: 0.7133
Epoch 5/50
19/19 [==============================] - 0s 7ms/step - loss: 0.4769 - accuracy: 0.7750
Epoch 6/50
19/19 [==============================] - 0s 7ms/step - loss: 0.4981 - accuracy: 0.7717
Epoch 7/50
19/19 [==============================] - 0s 7ms/step - loss: 0.4554 - accuracy: 0.8000
Epoch 8/50
19/19 [==============================] - 0s 6ms/step - loss: 0.4570 - accuracy: 0.8067
Epoch 9/50
19/19 [==============================] - 0s 7ms/step - loss: 0.3927 - accuracy: 0.8167
Epoch 10/50
19/19 [==============================] - 0s 7ms/step - loss: 0.3814 - accuracy: 0.8367
Epoch 11/50
19/19 [

In [24]:
_, acc = cnn.evaluate(test_data, test_labels)

9/9 [==============================] - 0s 10ms/step - loss: 0.8429 - accuracy: 0.7862
